In [2]:
import tensorflow as tf
import numpy as np
import PIL.Image
from pathlib import Path

from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input

import matplotlib.image as mpimg
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload

In [49]:
def load_image(path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)
    
    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    
    return img


def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

In [4]:
content_image_path = 'images/content_dog.jpg'

In [7]:
content = mpimg.imread(content_image_path)
content_preprocessed = preprocess_input(content[np.newaxis,...])
type(content)

numpy.ndarray

In [57]:
content.astype(np.float32).shape

(3024, 4032, 3)

In [63]:
image = content

max_ = np.max(image.shape)
scaling_factor = max_/512.0
target_shape = [int(i/scaling_factor) for i in image.shape[:-1]] + [3]

print(f'Preprocessing image from {image.shape} to {target_shape}')
image  = tf.image.resize(image, target_shape[:-1])

image.shape


Preprocessing image from (3024, 4032, 3) to [384, 512, 3]


TensorShape([384, 512, 3])

In [47]:
class NSTModel():
    
    def __init__(self, content_layers: list, style_layers: list):
        
        base_model = VGG19(include_top=False, weights='imagenet')
        base_model.trainable = False

        if content_layers is None:
            content_layers = ['block5_conv2']

        if style_layers is None:
            style_layers = [
                'block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1'
            ]

        content_outputs = []
        style_outputs = []
    
        self.content_layers = content_layers
        self.style_layers = style_layers
    
        for layer in base_model.layers:
            if layer.name in self.content_layers:
                content_outputs.append(layer.output)
            if layer.name in self.style_layers:
                style_outputs.append(layer.output)
    
        outputs = {'content': content_outputs, 'style': style_outputs}
        model = tf.keras.Model(inputs=base_model.inputs, outputs=outputs)
    
        self.nst_model = model

    def process(self, image: tf.Variable) -> tf.Tensor:

        # THIS IS EXPERIMENTAL:
        image = image/255.0
        image = preprocess_input(image)
        # END EXPERIMENTAL
        
        outputs = self.nst_model(image)

        return outputs

model = NSTModel(content_layers = ['block5_conv2'], style_layers=['block1_conv1'])

In [74]:
model.process(content_image)

{'content': [<tf.Tensor: shape=(1, 24, 32, 512), dtype=float32, numpy=
  array([[[[0.0000000e+00, 0.0000000e+00, 7.9760490e+01, ...,
            0.0000000e+00, 4.7501972e+01, 0.0000000e+00],
           [0.0000000e+00, 0.0000000e+00, 1.9511738e+01, ...,
            0.0000000e+00, 5.5931370e+01, 0.0000000e+00],
           [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
            0.0000000e+00, 4.8606441e+01, 0.0000000e+00],
           ...,
           [0.0000000e+00, 0.0000000e+00, 6.1945105e+00, ...,
            0.0000000e+00, 5.1038025e+01, 0.0000000e+00],
           [0.0000000e+00, 0.0000000e+00, 2.7994299e+01, ...,
            0.0000000e+00, 4.3557507e+01, 0.0000000e+00],
           [0.0000000e+00, 0.0000000e+00, 5.2255824e-02, ...,
            0.0000000e+00, 5.4374168e+01, 0.0000000e+00]],
  
          [[0.0000000e+00, 0.0000000e+00, 6.0511551e+01, ...,
            0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
           [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
       

In [5]:
vgg19 = VGG19()
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [26]:
def replace_max_pooling(model):

    layers = [l for l in model.layers]

    x = layers[0].output
    for i in range(1, len(layers)):
        if 'MaxPooling' in str(layers[i]):
            x = tf.keras.layers.AveragePooling2D(2)(x)
        else:
            x = layers[i](x)

    new_model = tf.keras.Model(inputs=model.inputs, outputs=x)
    return new_model

In [27]:
my_model = replace_max_pooling(vgg19)
my_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
average_pooling2d_42 (Averag (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
average_pooling2d_43 (Averag (None, 56, 56, 128)       0     

In [21]:
a = np.array([[1.0, 2.0, 3.0], [7, 6, 8]], dtype=np.float32)

tf.convert_to_tensor(a, dtype=tf.float32)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [7., 6., 8.]], dtype=float32)>

In [23]:
np.max(a)

8.0

In [74]:
tf.abs(content_image)

<tf.Tensor: shape=(1, 511, 434, 3), dtype=float32, numpy=
array([[[[0.8754116 , 0.8754116 , 0.9146273 ],
         [0.882353  , 0.882353  , 0.9215687 ],
         [0.87662715, 0.87662715, 0.91584283],
         ...,
         [0.8641181 , 0.8895097 , 0.9215687 ],
         [0.882353  , 0.882353  , 0.9215687 ],
         [0.86274517, 0.89019614, 0.9215687 ]],

        [[0.882353  , 0.882353  , 0.9215687 ],
         [0.882353  , 0.882353  , 0.9215687 ],
         [0.882353  , 0.882353  , 0.9215687 ],
         ...,
         [0.8641181 , 0.8895097 , 0.9215687 ],
         [0.882353  , 0.882353  , 0.9215687 ],
         [0.86274517, 0.89019614, 0.9215687 ]],

        [[0.882353  , 0.882353  , 0.9215687 ],
         [0.882353  , 0.882353  , 0.9215687 ],
         [0.8768519 , 0.8851036 , 0.9215687 ],
         ...,
         [0.8723704 , 0.8861752 , 0.920985  ],
         [0.875684  , 0.8856875 , 0.9215687 ],
         [0.87353253, 0.8876456 , 0.9223526 ]],

        ...,

        [[0.10302038, 0.09909882, 